In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from utils import clean_target
from categorical_ordinal import get_categorical_ordinal_columns
from categorical_nominal import get_categorical_nominal_columns
from columns_transformers import ColumnSelector, GetMunicipioFeatures
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import (OrdinalEncoder, OneHotEncoder, MinMaxScaler, PolynomialFeatures)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

<font color="orange"> <b>Grupos de features:</b></font>

- **Categorial Ordinal:**
    - TP_ (17-4-3 = 10)
    - Questions: ```["Q001", "Q002", "Q003", "Q004", "Q005","Q006", "Q007", "Q008", "Q009", "Q010", "Q011", "Q012", "Q013", "Q014", "Q015", "Q016", "Q017", "Q019", "Q022", "Q024"]``` (20)
                     
- **Categorial Nominal:**
    - IN_ : All Binary (52) 
    - TP_ : ```["TP_SEXO", "TP_ESTADO_CIVIL", "TP_COR_RACA", "TP_NACIONALIDADE"]``` (4)
    - SG_ : (4-1 = 3)
    - Questions: ```["Q018", "Q020", "Q021", "Q023", "Q025"]``` (5)
    
- **Numerical:**
    - NU_IDADE (1)

- Droped:
    - Identificator: ```[NU_INSCRICAO]``` (1)
    - More than 40% missing: ```['CO_ESCOLA', 'NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC']``` (4)
    - NO_M: (To many categories): ```['NO_MUNICIPIO_RESIDENCIA', 'NO_MUNICIPIO_NASCIMENTO', 'NO_MUNICIPIO_PROVA']``` (3)
    - NU_NOTA: Targets variables (5)

In [2]:
train_df = pd.read_parquet("data/train.parquet")
clean_target(train_df)
#test= pd.read_parquet("data/test.parquet")

In [3]:
categorical_ordinal_columns = get_categorical_ordinal_columns(train_df)
qtd_categorical_ordinal_columns=len(categorical_ordinal_columns)
print(f"Number of categorial ordinal features: {qtd_categorical_ordinal_columns}")

Number of categorial ordinal features: 30


In [4]:
categorical_nominal_columns = get_categorical_nominal_columns(train_df)
qtd_categorical_nominal_columns = len(categorical_nominal_columns)
print(f"Number of categorial nominal features: {qtd_categorical_nominal_columns}")

Number of categorial nominal features: 64


In [5]:
drop_columns = ["NU_INSCRICAO", 
                "CO_ESCOLA", 
                "NO_MUNICIPIO_ESC", 
                "SG_UF_ESC", 
                "TP_DEPENDENCIA_ADM_ESC", 
                "TP_LOCALIZACAO_ESC", 
                "TP_SIT_FUNC_ESC"]
qtd_drop_columns = len(drop_columns)
print(f"Number of columns dropped: {qtd_drop_columns}")

custom_features = ["NO_MUNICIPIO_RESIDENCIA", "NO_MUNICIPIO_NASCIMENTO", "NO_MUNICIPIO_PROVA"]

Number of columns dropped: 7


In [6]:
target_columns = train_df.filter(regex="NU_NOTA").columns.tolist()
qtd_target_columns = len(target_columns)
print(f"Number of targets: {qtd_target_columns}")

Number of targets: 5


In [7]:
numerical_columns = ["NU_IDADE"]
qtd_numerical_columns = len(numerical_columns)
print(f"Number of targets: {qtd_numerical_columns}")

Number of targets: 1


In [8]:
target_columns = train_df.filter(regex="NU_NOTA").columns.tolist()
qtd_target_columns = len(target_columns)
print(f"Number of targets: {qtd_target_columns}")

Number of targets: 5


In [9]:
all_columns = drop_columns + categorical_nominal_columns + categorical_ordinal_columns + numerical_columns + target_columns
qtd_total = qtd_drop_columns + qtd_categorical_nominal_columns + qtd_categorical_ordinal_columns + qtd_numerical_columns + qtd_target_columns
print(f"Total columns: {qtd_total}")

Total columns: 107


## **Create Pipeline**

In [10]:
"""
Variáveis categóricas com dados ordinais que tem dados faltantes:
- TP_ENSINO: Suposto que NaN representa a categoria faltante descrita nos metadados.
- TP_STATUS_REDACAO: Mapeado para outra classe (Faltou na prova)
"""
categorical_ordinal_pipe = Pipeline([
    ('selector', ColumnSelector(categorical_ordinal_columns)),
    ('imputer', SimpleImputer(missing_values=np.nan, 
                              strategy='constant', 
                              fill_value=0)),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
    ('extra_features', PolynomialFeatures(degree=2))
])

"""
Variáveis categóricas com dados ordinais que tem dados faltantes:
- SG_UF_NASCIMENTO: Mapeado para uma nova categoria
"""
categorical_nominal_pipe = Pipeline([
    ('selector', ColumnSelector(categorical_nominal_columns)),
    ('imputer', SimpleImputer(missing_values=np.nan, 
                              strategy='constant', 
                              fill_value="missing")),
    ('encoder', OneHotEncoder(drop="first", handle_unknown='ignore'))
])

numerical_pipe = Pipeline([
    ('selector', ColumnSelector(numerical_columns)),
    ('imputer', SimpleImputer(missing_values=np.nan, 
                              strategy='constant', 
                              fill_value=0)),
    ('scaler', MinMaxScaler())
])

extra_features_pipe = Pipeline([
    ('selector', ColumnSelector(custom_features)),
    ('extra_features', GetMunicipioFeatures())
])

preprocessor = FeatureUnion([
    ('categorical_ordinal', categorical_ordinal_pipe),
    ('categorical_nominal', categorical_nominal_pipe),
    ('numerical', numerical_pipe),
    ('municipio_features', extra_features_pipe)
])

kwargs_regressor = {"kernel":"poly",
                    "degree":3,
                    "C":1,
                    "verbose":2}

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', SVR(**kwargs_regressor))
])

In [11]:
def presence_filter(df, key):
    cond = df.filter(regex=f"PRESENCA_{key}|STATUS_{key}").iloc[:,0] == 1
    return df.loc[cond, :], df.loc[~cond, :]


y_structure = {"CN":[], 
               "CH":[],
               "LC":[],
               "MT":[],
               "REDACAO":[]}

In [ ]:
from joblib import dump
n_samples = 20000
sample = train_df.sample(n_samples)
    
for key, ys in tqdm(y_structure.items()):
    filtered_sample, missed_exam = presence_filter(sample, key)
    X = filtered_sample.drop(columns=target_columns+drop_columns)
    y = filtered_sample.filter(regex=f"NU_NOTA_{key}")
    
    X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.1, random_state=42)
    pipe.fit(X_train, y_train.values.ravel())
    dump(pipe, f"models/model_{key}.joblib") 
    y_train_hat = pipe.predict(X_train)
    y_test_hat = pipe.predict(X_test)
    ys.extend([y_train, y_test, y_train_hat, y_test_hat])

  0%|                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

[LibSVM]...........*...*.....*
optimization finished, #iter = 19036
obj = -739562.738191, rho = -455.246247
nSV = 13057, nBSV = 12947


 20%|███████████████████████████████▏                                                                                                                            | 1/5 [03:29<13:59, 209.98s/it]

[LibSVM]............*......*...*
optimization finished, #iter = 20610
obj = -840947.215204, rho = -495.889126
nSV = 13834, nBSV = 13722


 40%|██████████████████████████████████████████████████████████████▍                                                                                             | 2/5 [07:11<10:50, 216.73s/it]

[LibSVM]................*...*...*
optimization finished, #iter = 21885
obj = -627078.747080, rho = -516.483630
nSV = 13829, nBSV = 13692


 60%|█████████████████████████████████████████████████████████████████████████████████████████████▌                                                              | 3/5 [10:51<07:16, 218.11s/it]

[LibSVM]...........*
optimization finished, #iter = 11194
obj = -1035577.486263, rho = -483.186493
nSV = 13061, nBSV = 12972


 80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                               | 4/5 [14:19<03:34, 214.28s/it]

[LibSVM]..........*
optimization finished, #iter = 10333
obj = -1508419.501308, rho = -574.614782
nSV = 13308, nBSV = 13223


In [ ]:
mean_train = []
mean_test = []
for key, ys in tqdm(y_structure.items()):
    train_error = mean_squared_error(ys[0], ys[2], squared=False)
    mean_train.append(train_error)
    test_error = mean_squared_error(ys[1], ys[3], squared=False)
    mean_test.append(test_error)
    print(key)
    print(f"Train: {train_error}")
    print(f"Test: {test_error}\n")

In [ ]:
np.mean(mean_train)

In [ ]:
np.mean(mean_test)